<a href="https://colab.research.google.com/github/rjakkani1015/rashmi_INFO5731_-Fall2021/blob/main/INFO5731_Assignment_Two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Two**

In this assignment, you will try to gather text data from open data source via web scraping or API. After that you need to clean the text data and syntactic analysis of the data.

# **Question 1**

(20 points). Write a python program to collect text data from **either of the following sources** and save the data into a **csv file**:

(1) Collect all the customer reviews of the product [Apple iPhone 11](https://www.amazon.com/Apple-iPhone-11-64GB-Unlocked/dp/B07ZPKF8RG/ref=sr_1_13?dchild=1&keywords=iphone+12&qid=1631721363&sr=8-13) on amazon.

(2) Collect all User Reviews of the film [Shang-Chi and the Legend of the Ten Rings](https://www.imdb.com/title/tt9376612/reviews?ref_=tt_sa_3) from IMDB.

(3) Collect all the abstracts research papers by using the query [smart health](https://citeseerx.ist.psu.edu/search?q=natural+language+processing&submit.x=0&submit.y=0&sort=rlv&t=doc) from CiteSeerX.

(4) Collect the top 10,000 tweets by using hashtag ["#blacklivesmatter"](https://twitter.com/hashtag/blacklivesmatter) from Twitter with the following requirements: Location (Texas), Time frame (2021-01-01 to 2021-09-01). 


In [ ]:
from bs4 import BeautifulSoup
import requests, lxml, os, json
from dateutil.parser import parse
import pandas as pd

proxies = {
    'http': os.getenv('HTTP_PROXY')  # or just type proxy here without os.getenv()
}

headers = {
    'User-agent':
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582"
}

df = pd.DataFrame()

counter = 0

for i in range(0, 112):
    start = str(i * 10)

    if i == 0:
        url = 'https://citeseerx.ist.psu.edu/search?q=smart+health&submit.x=0&submit.y=0&sort=rlv&t=doc'
    else:
        url = 'https://citeseerx.ist.psu.edu/search?q=smart+health&t=doc&sort=rlv&start=' + start

    html = requests.get(url, headers=headers, proxies=proxies).text
    soup = BeautifulSoup(html, 'lxml')

    for i in soup.findAll("div", {'class': "result"}):
        abstract = i.find("div", {'class': "pubabstract"}).text

        d = {
            'abstract': abstract
        }

        df = df.append(d, ignore_index=True)


df.to_csv("citeseer")
print("500 records download successful to Google Colab folders")


500 records download successful to Google Colab folders


# **Question 2**

(30 points) Here is a [legal case](https://github.com/unt-iialab/info5731-fall2021/blob/main/assignments/01-05-1%20%20Adams%20v%20Tanner.txt) we collected from westlaw, please follow the steps we mentioned in lesson 5 to clean the data:



(1) Basic feature extraction using text data

*   Number of sentences
*   Number of words
*   Number of characters
*   Average word length
*   Number of stopwords
*   Number of special characters
*   Number of numerics
*   Number of uppercase words

(2) Basic Text Pre-processing of text data

*   Lower casing
*   Punctuation removal
*   Stopwords removal
*   Frequent words removal
*   Rare words removal
*   Spelling correction
*   Tokenization
*   Stemming
*   Lemmatization

(3) Save all the **clean sentences** to a **csv file** (one column, each raw is a sentence) after finishing all the steps above.


(4) Advance Text Processing

*   Calculate the term frequency of all the terms.
*   Print out top 10 1-gram, top 10 2-grams, and top 10 3-grams terms as features.


In [ ]:
from nltk import PorterStemmer
from nltk.corpus import stopwords
import numpy as np
import nltk

from urllib.request import urlopen
import pandas as pd

from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('punkt')

data_url = "https://raw.githubusercontent.com/unt-iialab/info5731-fall2021/main/assignments/01-05-1%20%20Adams%20v%20Tanner.txt"
legal_case_data = urlopen(data_url)

data_input = pd.DataFrame()

for line in legal_case_data:
    extract = line.decode('utf-8').encode("ascii", "ignore").rstrip()
    d = {
        'extract': extract
    }
    data_input = data_input.append(d, ignore_index=True)
  
data_input['avg_word'] = data_input['extract'].apply(lambda x: avg_word(x))
data_input[['extract', 'avg_word']].head()


data_input['word_count'] = data_input['extract'].apply(lambda x: len(str(x).split(" ")))
data_input[['extract', 'word_count']].head()


data_input['char_count'] = data_input['extract'].str.len()
data_input[['extract', 'char_count']].head()

def avg_word(sentence):
    if len(sentence) > 0:
        words = sentence.split()
        return (sum(len(word) for word in words)/len(words))
    else:
        return 0


nltk.download('stopwords')
stop = stopwords.words('english')

data_input['stopwords'] = data_input['extract'].apply(lambda x: len([x for x in x.split() if x in stop]))
data_input[['extract', 'stopwords']].head()

data_input['hashtags'] = data_input['extract'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
data_input[['extract', 'hashtags']].head()

data_input['numerics'] = data_input['extract'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
data_input[['extract', 'numerics']].head()

data_input['upper'] = data_input['extract'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
data_input[['extract', 'upper']].head()


data_input['extract'] = data_input['extract'].apply(lambda x: " ".join(x.lower() for x in x.split()))
data_input['extract'].head()

data_input['extract'] = data_input['extract'].str.replace('[^\w\s]', '')
data_input['extract'].head()


stop = stopwords.words('english')
data_input['extract'] = data_input['extract'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
data_input['extract'].head()

freq = pd.Series(' '.join(data_input['extract']).split()).value_counts()[:10]
freq

freq = list(freq.index)
data_input['extract'] = data_input['extract'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
data_input['extract'].head()


freq = pd.Series(' '.join(data_input['extract']).split()).value_counts()[-10:]
freq

freq = list(freq.index)
data_input['extract'] = data_input['extract'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
data_input['extract'].head()

data_input['extract'][:5].apply(lambda x: str(TextBlob(x).correct()))





TextBlob(data_input['extract'][1]).words

st = PorterStemmer()

data_input['extract'][:5].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))


nltk.download('wordnet')

data_input['extract'] = data_input['extract'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

data_input['extract'].head()

TextBlob(data_input['extract'][0]).ngrams(2)


tf1 = (data_input['extract'][1:2]).apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).reset_index()
tf1.columns = ['words','tf']
tf1




for i,word in enumerate(tf1['words']):
  tf1.loc[i, 'idf'] = np.log(data_input.shape[0] / (len(data_input[data_input['extract'].str.contains(word)])))

tf1


tf1['tfidf'] = tf1['tf'] * tf1['idf']
tf1


myvocabulary = ['life', 'learning', 'game']
corpus = {1: "The game of life is a game of everlasting learning", 2: "The unexamined life is not worth living", 3: "Never stop learning"}
tfidf = TfidfVectorizer(vocabulary = myvocabulary, max_features=1000, lowercase=True, analyzer='word', stop_words='english', ngram_range=(1,3))
tfs = tfidf.fit_transform(corpus.values())
feature_names = tfidf.get_feature_names()
corpus_index = [n for n in corpus]

df = pd.DataFrame(tfs.T.todense(), index=feature_names, columns=corpus_index)
print(df)

bow = CountVectorizer(max_features=50, lowercase=True, ngram_range=(1,3),analyzer="word")
train_bow = bow.fit_transform(data_input['extract'])
bow.get_feature_names()






# **Question 3**

(30 points). Write a python program to conduct **syntax and structure analysis** of the clean text you just saved above. The syntax and structure analysis includes: 

(1) Parts of Speech (POS) Tagging: Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [ ]:
from urllib.request import urlopen
import pandas as pd
import spacy

target_url = "https://raw.githubusercontent.com/unt-iialab/info5731-fall2021/main/assignments/01-05-1%20%20Adams%20v%20Tanner.txt"
legal_case_data = urlopen(target_url)

data = pd.DataFrame()

for line in legal_case_data:
  extract = line.decode('utf-8').encode("ascii", "ignore").rstrip()
  d = {
    'extract': extract
  }
  data = data.append(d, ignore_index=True)

data_str = str(legal_case_data)


sp = spacy.load("en_core_web_sm")
extracts = sp(data_str)

for word in extracts:
  print(word.text + "\t \t" + str(word.pos))

all_pos = extracts.count_by(spacy.attrs.POS)

for k,v in all_pos.items():
  print(extracts.vocab[k].text + " \t" + str(v))

for entity in extracts:
  print(entity.text + "\t\t" + entity.label)

**Write your explanations of the constituency parsing tree and dependency parsing tree here (Question 3-2):**




In [ ]:
# You explaination here





# **Question 4**

(20 points) Python Regular Expression.

(1) Write a Python program to remove leading zeros from an IP address.

ip = "260.08.094.109"


(2) Write a Python Program to extract all the years from the following sentence.

sentence = "The 2010s were a dramatic decade, filled with ups and downs, more than 1000 stroies have happened. As the decade comes to a close, Insider took a look back at some of the biggest headline-grabbing stories, from 2010 to 2019. The result was 119 news stories that ranged from the heartwarming rescue of a Thai boys' soccer team from a flooded cave to the divisive election of President Donald Trump."

In [ ]:
import re

ip_address = input("enter ip address:")
address = ip_address.split(".") ## spliting using the split() functions
address = [int(part) for part in address] ## converting every part to int
address = [str(part) for part in address] ## converting each to str again before joining them
ip_address = ".".join(address) ## joining every part using the join() method
print("ip address without leading zeros:", ip_address)

s = '''The 2010s were a dramatic decade, filled with ups and downs, more than 1000 stroies have happened.
    As the decade comes to a close, Insider took a look back at some of the biggest headline-grabbing stories, from 2010 to 2019.
    The result was 119 news stories that ranged from the heartwarming rescue of a Thai boys soccer team from a flooded
    cave to the divisive election of President Donald Trump'''

output = re.findall(r"(2+\d{3})", s)
print("All the years from the sentence:", output)






enter ip address:260.08.094.109
ip address without leading zeros: 260.8.94.109
All the years from the sentence: ['2010', '2010', '2019']
